# Floodplain and terrace extraction

Notebook last updated by Simon M Mudd 07/12/2022

In this notebook we will extract some terrace and floodplains using the method described in Clubb et al, ESURF, 2017 (https://esurf.copernicus.org/articles/5/369/2017/). 

## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **Add some necessary packages** section. 

In [ ]:
First we install `lsdviztools`. 

Note that if we install `condacolab` first the `lsdviztools` installation fails. 

In [ ]:
!pip install lsdviztools

Now we need to install lsdtopotools. We do this using something called `condacolab`. 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Now install `lsdtopotools`. The `&> /dev/null` bit just stops a bunch of rubbish being printed to the screen.

In [ ]:
!mamba install lsdtopotools &> /dev/null

## Add some necessary packages

First check the version of lsdviztools

In [ ]:
import lsdviztools
lsdviztools.__version__

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

You can also check the version of lsdtt. This notebook was made with version *0.7*

In [ ]:
!lsdtt-basic-metrics -v

### Get the example data

The example data is an 18 Mb topographic dataset which you need to download:

In [ ]:
# Get the data from github
import urllib
urllib.request.urlretrieve("https://raw.githubusercontent.com/LSDtopotools/ExampleTopoDatasets/master/ChannelExtractionData/IndianCreek_1m/indian_creek.hdr", "indian_creek.hdr")
urllib.request.urlretrieve("https://raw.githubusercontent.com/LSDtopotools/ExampleTopoDatasets/master/ChannelExtractionData/IndianCreek_1m/indian_creek.bil", "indian_creek.bil")

Lets make sure that file (`indian_creek.bil` and its header) have been downloaded:

In [ ]:
!ls indian_creek*

## Look at the topography

Lets start by looking at the topography:

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true"}
r_prefix = "indian_creek"
w_prefix = "indian_creek"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
### Plot the hillshade
%matplotlib inline
Base_file = "indian_creek"
DataDirectory = "./"
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology")

We can get the channel sources using using `lsdtt-channel-extraction`. We will use the `print_wiener_channels` option. You also need to get the sources with the option `print_sources_to_csv`. This will take around a minute. 

In [ ]:
lsdtt_parameters = {"print_wiener_channels" : "true",
                    "print_sources_to_csv" : "true"}
r_prefix = "indian_creek"
w_prefix = "indian_creek"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-channel-extraction", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

Right, now that that has finished, lets see what files we got:

In [ ]:
!ls indian_creek*

There are actually 2 files with sources here, one is `indian_creek_ATsources.csv` and the other is `indian_creek_Wsources.csv`. `AT` stands for *area threshold* (and is produced by default) and `W` is for wiener. 

## Look at the Sources using folium

We can plot these sources on a folium map to compare them.
Before we do that though, we need to load the two datasets with pandas

In [ ]:
import pandas as pd

sources_AT_df = pd.read_csv("indian_creek_ATsources.csv")
sources_W_df = pd.read_csv("indian_creek_Wsources.csv")

In [ ]:
# This is for the area threshold points

import folium

#create a map
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Terrain')

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=2,
                        weight=5).add_to(this_map)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
sources_AT_df.apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file
this_map.save('simple_dot_plot.html')

this_map

In [ ]:
# This is for the wiener points

import folium

#create a map
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Terrain')

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=2,
                        weight=5).add_to(this_map)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
sources_W_df.apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file
this_map.save('simple_dot_plot.html')

this_map

You will notice that the Wiener sources appear at much more realistic locations on this map. If you want to use channel sources and are particular about the headwaters, you probably want to use this method (see https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2013WR015167 and https://esurf.copernicus.org/articles/4/627/2016/).

When looking for valleys and floodplains, however, you can set a minimum stream order so that the smallest channels are not captured. Even if the area threshold channel sources are not particularly accurate, the higher order streams will still be in reasonable locations. 

## Get some floodplains

This needs a few updates before it is working. Bear with me...

In [ ]:


lsdtt_parameters = {"CHeads_file" : "indian_creek_ATsources",
                    "fill_floodplain" : "true"}
r_prefix = "indian_creek"
w_prefix = "indian_creek"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-valley-metrics", 
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

Now lests see what we have:

In [ ]:
!ls indian_creek*

In [ ]:
ls *.driver

In [ ]:
!lsdtt-valley-metrics Test_01.driver

In [ ]:
!ls indian_creek*